In [1]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
import pandas as pd

In [ ]:
# Treiber laden
driver = webdriver.Firefox()
wait = WebDriverWait(driver, 10)

In [ ]:

# Seite laden
driver.get("https://www.bmw-berlin-marathon.com/impressionen/statistik-und-geschichte/ergebnisarchiv/")
time.sleep(3)
# Cookies bestaetigen
driver.find_element_by_xpath("//button[@aria-label='Akzeptiere alle']").click()
time.sleep(2)

In [ ]:
# Jahre die geparsed werden sollen
years = list(range(2007,2020))

In [ ]:
# Alle Jahre mit den internen Values des Dropdownfeldes ermittelnt
# Die Values werden benoetigt um spaeter die jeweiligen Jahr auszuwaehlen
event = driver.find_element_by_xpath("//div[@id='scc_timing_21058']//select[@class='events border-input']")
options = event.find_elements_by_tag_name("option")
# Nur Values zu den ausgeweahlten Jahren ermitteln
dict_events = {}
for option in options:
    year = int(option.text[0:5])
    if year in years:
        years.remove(year)
        dict_events[year] = option.get_attribute("value")
    if len(years) == 0:
        break

In [ ]:
# Leeres Dataframe anlegen
df_results = pd.DataFrame(pd.DataFrame(
    columns=['Jahr', 'Platz','Vorname', 'Nachname', 'NAT', 'SN', 'Verein','Geschlecht','Netto', 'Brutto','5KM','10KM','15KM','20KM','HM','25KM','30KM','35KM','40KM'])
    ,dtype=str)

In [ ]:
# Ueber die Jahre iterieren und Werte speichern
for year in dict_events:
    # Jahr auswaehlen
    xpath_event_value = "//option[@value='" + dict_events[year] + "']"
    event.find_element_by_xpath(xpath_event_value).click()
    time.sleep(3)
    # Geschlecht filtern
    for ges in ['M', 'W']:
        # Filter - Geschlecht setzen
        event.find_element_by_xpath("//table[1]/thead[1]/tr[@class='filter']/th[8]/input[1]").send_keys(ges)
        time.sleep(3)
        # Ueber die Ergebnisse iterieren: Anzahl der Ergebniss x 2:
        for row in range(1,20,2):
            xpath_result_row = "//table[1]/tbody[1]/tr[" + str(row) + "]"
            xpath_result_row_zz = xpath_result_row + "/td[1]"
            # Gesamtzeiten + Alle Attribute speichern
            row_temp = {
                'Jahr' : year
                ,'Platz' : event.find_element_by_xpath(xpath_result_row + "/td[2]").text
                ,'Vorname' : event.find_element_by_xpath(xpath_result_row + "/td[3]").text
                ,'Nachname' : event.find_element_by_xpath(xpath_result_row + "/td[4]").text
                ,'NAT' : event.find_element_by_xpath(xpath_result_row + "/td[5]").text
                ,'SN' : event.find_element_by_xpath(xpath_result_row + "/td[6]").text
                ,'Verein' : event.find_element_by_xpath(xpath_result_row + "/td[7]").text
                ,'Geschlecht' : event.find_element_by_xpath(xpath_result_row + "/td[8]").text
                ,'Netto' : event.find_element_by_xpath(xpath_result_row + "/td[9]").text
                ,'Brutto' : event.find_element_by_xpath(xpath_result_row + "/td[10]").text
            }
            #print(row_temp)
            # Zwischnezeiten anzeigen    
            event.find_element_by_xpath(xpath_result_row_zz).click()
            time.sleep(3)
            # Zwischenzeiten zum Gesamtergebniss hizufuegen
            for zzeit in range(1,10):
                zz_path_desc = "//table[1]/tbody[1]/tr[" + str(row+1) + "]/td[1]/dl[1]/dd[1]/ul[1]/li[" + str(zzeit) + "]/span[1]"
                zz_path_value = "//table[1]/tbody[1]/tr[" + str(row+1) + "]/td[1]/dl[1]/dd[1]/ul[1]/li[" + str(zzeit) + "]/span[2]"
                row_temp.update({
                    event.find_element_by_xpath(zz_path_desc).text.upper() : event.find_element_by_xpath(zz_path_value).text
                })
                #print(event.find_element_by_xpath(zz_path_desc).text + " | " + event.find_element_by_xpath(zz_path_value).text)
            # Ins Dataframe speichern
            df_results = df_results.append(row_temp, ignore_index=True)
        # Filter - Geschlecht entfernen
        event.find_element_by_xpath("//table[1]/thead[1]/tr[@class='filter']/th[8]/input[1]").clear()
        time.sleep(3)

    

In [ ]:
# Ergebnisse ausgeben
# print(df_results.head())

In [ ]:
# Treiber schliessen
driver.close()

In [ ]:
# Daten in die CSV Exportieren
file_path = '/home/paul/Downloads/DatenMA/Berlin/'
file_name = 'daten_wmm_berlin.csv'

df_results.to_csv(file_path+file_name, sep=';', index=False)